In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.collections import LineCollection

from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.utils import check_random_state
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AR
from datetime import datetime


data = pd.read_csv('newfile_winter.csv')
gold=pd.DataFrame(data[['Country','Year','Gold']])
date=pd.DataFrame(data['Year'])


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.collections import LineCollection

from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.utils import check_random_state
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AR


# Importing the Dataset

data = pd.read_csv('newfile_summer.csv')
gold=pd.DataFrame(data[['Country','Year','Gold']])

# Getting number of countries
country=pd.DataFrame(data['Country'])
country_cnt = len(country['Country'].value_counts())

#####################################################################################
# Creating a loop for each country in the dataframe

Total_time = []
Total_prediction = []
country_included = []
country_record = []
index = 0
country_name = gold.iloc[index][0]

while (len(country_record)<country_cnt):
    year = []
    gold_cnt = []
    gold_cnt_1 = []
    while (country_name in country_record):
        index += 1
        country_name = gold.iloc[index][0]    
    country_record.append(country_name)
    
    print ("*********************************************************************************")
    print ('Country: ', country_name)
    
    #####################################################################################
    # Extracting gold and year data for the country, excluding year 1906 cos its not a multiple of 4 
    
    for i in range(len(gold)):
        if (gold.iloc[i][0] == country_name):
            if (gold.iloc[i][1]!=1906):
                year.append(gold.iloc[i][1])
                gold_cnt.append(gold.iloc[i][2])
                gold_cnt_1.append(gold.iloc[i][2])
    
    #####################################################################################
    # Calculating the missing values for years
    
    n = len(year)
    missing_years = []
    for i in range(n-1):
        if (year[i+1]-year[i]>4):
            curr = year[i]+4
            fin = year[i+1]
            while (curr<fin):
                missing_years.append(curr)
                curr += 4
                
    if (len(year)<20):
        print("Country has less than 20 entries, skipping...")
        continue
    
    No_Medals = True
    for i in gold_cnt:
        if (i!=0):
            No_Medals = False
    if (No_Medals == True):
        print("Country has never gotten a medal, skipping...")
        continue
    
    #####################################################################################
    # Converting python array format into numpy array format

    x = np.array(year)
    x_missing = np.array(missing_years)
    y = np.array(gold_cnt_1)
    y_1 = np.array(gold_cnt_1)
    
    ###############################################################################
    # Fit IsotonicRegression and LinearRegression models

    ir = IsotonicRegression()

  
    y_ = ir.fit_transform(x, y)

    lr = LinearRegression()
    lr.fit(x[:, np.newaxis], y_1)  # x needs to be 2d for LinearRegression
    
    ###############################################################################
    # Plotting initial data
    
    segments = [[[i, y[i]], [i, y_[i]]] for i in range(n)]
    lc = LineCollection(segments, zorder=0)
    lc.set_array(np.ones(len(y)))
    lc.set_linewidths(np.full(n, 1))
    p=(255,255,0)
    fig = plt.figure()
    plt.plot(x, y, 'b.', markersize=12)
    plt.plot(x, y_, 'y.-', markersize=12)
    plt.plot(x, lr.predict(x[:, np.newaxis]), 'r-')
    plt.gca().add_collection(lc)
    plt.legend(('Data', 'Isotonic Fit', 'Linear Fit'), loc='upper left')
    plt.title(country_name)
    
    print("Before: ")
    plt.show()
    
   
    ###############################################################################
    # Predicting gold medal count for missing years and filling in data
    
    x_missing_1 = x_missing.reshape(-1,1)
    y_missing_1 = lr.predict(x_missing_1).tolist() 
    
    y_missing = ir.predict(x_missing).tolist()
    
    for i in range(len(missing_years)):
        insert = 0
        while (missing_years[i]-year[insert]>0):
            insert += 1
        year.insert(insert, missing_years[i])
        if (y_missing[i]<0):
            y_missing[i]=0
        gold_cnt.insert(insert, round(y_missing[i]))
        
    for i in range(len(missing_years)):
        insert = 0
        while (missing_years[i]-year[insert]>0):
            insert += 1
        if (y_missing_1[i]<0):
            y_missing_1[i]=0
        gold_cnt_1.insert(insert, round(y_missing_1[i]))
    
    x_complete = np.array(year)
    y_complete = np.array(gold_cnt)
    y_complete_1 = np.array(gold_cnt_1)
    
    ###############################################################################
    # Fit new IsotonicRegression and LinearRegression models
    
    ir = IsotonicRegression()
    y_ = ir.fit_transform(x_complete, y_complete)

    lr = LinearRegression()
    lr.fit(x_complete[:, np.newaxis], y_complete_1)  # x needs to be 2d for LinearRegression
    
    ###############################################################################
    # Plotting smoothed out data
    
    segments = [[[i, y_complete[i]], [i, y_[i]]] for i in range(n)]
    lc = LineCollection(segments, zorder=0)
    lc.set_array(np.ones(len(y_complete)))
    lc.set_linewidths(np.full(n, 1))
    p=(255,255,0)
    fig = plt.figure()
    plt.plot(x_complete, y_complete, 'b.', markersize=12)
    plt.plot(x_complete, y_, 'y.-', markersize=12)
    plt.plot(x_complete, lr.predict(x_complete[:, np.newaxis]), 'r-')
    plt.gca().add_collection(lc)
    plt.legend(('Data', 'Isotonic Fit', 'Linear Fit'), loc='upper left')
    plt.title(country_name)

    print("After_1: ")
    plt.show()
    
    segments = [[[i, y_complete_1[i]], [i, y_[i]]] for i in range(n)]
    lc = LineCollection(segments, zorder=0)
    lc.set_array(np.ones(len(y_complete_1)))
    lc.set_linewidths(np.full(n, 1))
    p=(255,255,0)
    fig = plt.figure()
    plt.plot(x_complete, y_complete_1, 'b.', markersize=12)
    plt.plot(x_complete, y_, 'y.-', markersize=12)
    plt.plot(x_complete, lr.predict(x_complete[:, np.newaxis]), 'r-')
    plt.gca().add_collection(lc)
    plt.legend(('Data', 'Isotonic Fit', 'Linear Fit'), loc='upper left')
    plt.title(country_name)

    print("After: ")
    plt.show()
    